In [2]:
import os
os.chdir("../")

In [4]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_file_dir: Path
    local_file: Path

In [5]:
from PhishingDomainDetection.constants import CONFIG_FILE_PATH, PARAMS_FILE_APTH
from PhishingDomainDetection.utils import read_yaml_file, create_directories

In [7]:
from PhishingDomainDetection import log

In [8]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH):
        self.config = read_yaml_file(config_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_file_dir=config.local_file_dir,
            local_file=config.local_file
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import pandas as pd
import io
import requests

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self,):
        url = self.config.source_URL
        url_data = requests.get(url).content
        self.df = pd.read_csv(io.StringIO(url_data.decode('-utf-8')))
        return self.df

    def save_data(self):
        data_ingestion_root_dir = self.config.root_dir
        local_file_dir = self.config.local_file

        raw_local_dir_path = os.path.join(data_ingestion_root_dir, local_file_dir)

        self.df.to_csv(raw_local_dir_path, index=False)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.save_data()
except Exception as e:
    raise e 

[2023-02-03 19:25:55,579: INFO common]: yaml file configs\config.yaml load  successfully
[2023-02-03 19:25:55,580: INFO common]: created directory at artifacts
[2023-02-03 19:25:55,581: INFO common]: created directory at artifacts/data_ingestion
